# 🎙️ Voice Pipeline — Whisper STT + Phi-3.5 RAG + Edge-TTS
### Bengali AI Tutor | Group 11 — BUET CSE

**Full Voice Pipeline:**
```
🎤 Bengali Audio Input
        ↓
   Whisper STT (OpenAI)
        ↓
   Language Detection
        ↓
   FAISS RAG (NCTB context)
        ↓
   Phi-3.5 Mini (answer generate)
        ↓
   Edge-TTS (Bengali/English voice)
        ↓
🔊 Audio Output → Student শুনতে পায়
```

---
**⚠️ Setup:**
- আগের Phi-3 notebook এর output dataset হিসেবে add করো
- GPU T4 ON রাখো
- Internet ON রাখো

## Step 1: Install Libraries

In [ ]:
# Voice pipeline libraries
!pip install -q openai-whisper          # Whisper STT
!pip install -q edge-tts                # Microsoft Edge TTS (Bengali voice)
!pip install -q pydub                   # Audio processing
!pip install -q soundfile               # Audio file read/write
!pip install -q librosa                 # Audio analysis

# RAG pipeline libraries (আগের notebook থেকে)
!pip install -q sentence-transformers faiss-cpu
!pip install -q transformers accelerate bitsandbytes

# System dependencies
!apt-get install -q ffmpeg              # Audio conversion

print('✅ সব install হয়েছে!')

## Step 2: RAG Pipeline Load (FAISS + Phi-3.5)

In [ ]:
import os, pickle, torch
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ── Paths ──────────────────────────────────────────────────────
# আগের notebook এর output dataset add করার পর path:
BASE = '/kaggle/input/nctb-rag-output'  # ← dataset name অনুযায়ী change করো
if not os.path.exists(BASE):
    BASE = '/kaggle/working'
    print('⚠️ Working directory থেকে load হচ্ছে')

# FAISS load
print('🔄 FAISS index load হচ্ছে...')
INDEX = faiss.read_index(f'{BASE}/faiss_index/bangla_class3.faiss')
with open(f'{BASE}/faiss_index/chunks.pkl', 'rb') as f:
    CHUNKS = pickle.load(f)
print(f'✅ FAISS loaded — {INDEX.ntotal} vectors')

# Embedding model
print('🔄 Embedding model load হচ্ছে...')
EMBED_MODEL = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    device='cuda' if torch.cuda.is_available() else 'cpu'
)
print('✅ Embedding model ready!')

# Phi-3.5 load
print('\n🔄 Phi-3.5 Mini load হচ্ছে (~5 min)...')
MODEL_ID = 'microsoft/Phi-3.5-mini-instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

TOKENIZER = AutoTokenizer.from_pretrained(MODEL_ID)
PHI3 = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.float16,
)
PHI3.eval()
print(f'✅ Phi-3.5 ready! Memory: {torch.cuda.memory_allocated()//1024**2} MB')

## Step 3: Whisper STT Setup

**Whisper** — OpenAI এর speech recognition model।  
Bengali + English দুটোই recognize করে।  
**medium** model — accuracy ও speed এর balance।

In [ ]:
import whisper
import numpy as np
import soundfile as sf
import tempfile

print('🔄 Whisper model load হচ্ছে...')
# Model size options:
# 'tiny'   → fastest, less accurate
# 'base'   → good balance for simple Bengali
# 'medium' → best Bengali accuracy (recommended)
# 'large'  → most accurate but slow
WHISPER_MODEL = whisper.load_model('medium')
print('✅ Whisper medium loaded!')

def transcribe_audio(audio_path, language=None):
    """
    Audio file থেকে text transcribe করা।
    
    Args:
        audio_path: audio file path (.wav, .mp3, .ogg)
        language: 'bn' = Bengali force, None = auto detect
    
    Returns:
        dict: {text, language, confidence}
    """
    result = WHISPER_MODEL.transcribe(
        audio_path,
        language=language,        # None = auto detect
        task='transcribe',        # 'transcribe' = same language, 'translate' = to English
        fp16=torch.cuda.is_available(),
        verbose=False
    )
    
    detected_lang = result.get('language', 'unknown')
    text = result['text'].strip()
    
    # Confidence score (average of segments)
    segments = result.get('segments', [])
    if segments:
        avg_confidence = np.mean([-s.get('no_speech_prob', 0) for s in segments])
    else:
        avg_confidence = 0.0
    
    return {
        'text': text,
        'language': detected_lang,
        'confidence': float(avg_confidence)
    }

# Test with a synthetic audio (silence) — real audio দিয়ে test করবে
print('\n✅ Whisper STT function ready!')
print('   Supported: Bengali (bn), English (en), + 99 more languages')

## Step 4: Edge-TTS Setup (Bengali Voice)

**Edge-TTS** — Microsoft এর free TTS।  
Bengali voice: **bn-BD-NabanitaNeural** (female) বা **bn-BD-PradeepNeural** (male)

In [ ]:
import edge_tts
import asyncio
import IPython.display as ipd

# Available Bengali voices
VOICES = {
    'bengali_female': 'bn-BD-NabanitaNeural',   # বাংলাদেশ Bengali female
    'bengali_male':   'bn-BD-PradeepNeural',     # বাংলাদেশ Bengali male
    'english_female': 'en-US-JennyNeural',       # English fallback
}

async def text_to_speech_async(text, voice, output_path):
    """Async TTS generation."""
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(output_path)

def generate_speech(text, language='bengali', gender='female', output_path=None):
    """
    Text থেকে Bengali/English speech generate করা।
    
    Args:
        text: বলার text
        language: 'bengali' বা 'english'
        gender: 'female' বা 'male'
        output_path: save করার path (None = temp file)
    
    Returns:
        audio file path
    """
    if output_path is None:
        output_path = tempfile.mktemp(suffix='.mp3')
    
    # Voice select করা
    if language == 'bengali':
        voice = VOICES[f'bengali_{gender}']
    else:
        voice = VOICES['english_female']
    
    # Async run করা
    asyncio.run(text_to_speech_async(text, voice, output_path))
    
    return output_path

def play_audio(audio_path):
    """Kaggle notebook এ audio play করা।"""
    return ipd.Audio(audio_path, autoplay=True)

# Test Bengali TTS
print('🔄 Bengali TTS test হচ্ছে...')
test_text = 'আমি তোমার AI শিক্ষক। তোমার যেকোনো প্রশ্ন করো।'
test_audio = generate_speech(test_text, language='bengali', gender='female')
print(f'✅ Audio generated: {test_audio}')
print('\nAudio play হচ্ছে:')
play_audio(test_audio)

## Step 5: RAG + Generation Functions

In [ ]:
def retrieve(question, k=3):
    """NCTB থেকে relevant context retrieve করা।"""
    q_emb = EMBED_MODEL.encode(
        [question], normalize_embeddings=True, convert_to_numpy=True
    ).astype('float32')
    scores, indices = INDEX.search(q_emb, k)
    return [
        {'text': CHUNKS[i], 'score': float(s)}
        for s, i in zip(scores[0], indices[0]) if i >= 0
    ]

def detect_language(text):
    """Bengali নাকি English detect করা।"""
    bn_chars = sum(1 for c in text if '\u0980' <= c <= '\u09FF')
    return 'bengali' if bn_chars > len(text) * 0.15 else 'english'

def generate_answer(question, mode='chat', max_new_tokens=250):
    """RAG + Phi-3.5 দিয়ে উত্তর generate করা।"""
    chunks = retrieve(question, k=3)
    context = '\n\n'.join(
        f'[অংশ {i+1}]: {c["text"]}' for i, c in enumerate(chunks)
    )
    
    lang = detect_language(question)
    
    if lang == 'bengali':
        system = (
            'তুমি একটি AI শিক্ষক। তৃতীয় শ্রেণির ছাত্রছাত্রীদের '
            'NCTB পাঠ্যবই অনুযায়ী সহজ বাংলায় সাহায্য করো। '
            'উত্তর ছোট ও বন্ধুত্বপূর্ণ রাখো।'
        )
        user = f'পাঠ্যবইয়ের অংশ:\n{context}\n\nপ্রশ্ন: {question}'
    else:
        system = (
            'You are an AI tutor for Class 3 students in Bangladesh. '
            'Answer using NCTB textbook content in simple English.'
        )
        user = f'Textbook content:\n{context}\n\nQuestion: {question}'
    
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user',   'content': user}
    ]
    
    prompt = TOKENIZER.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = TOKENIZER(
        prompt, return_tensors='pt', truncation=True, max_length=2048
    ).to(PHI3.device)
    
    with torch.no_grad():
        output = PHI3.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.3,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=TOKENIZER.eos_token_id
        )
    
    generated = output[0][inputs['input_ids'].shape[1]:]
    answer = TOKENIZER.decode(generated, skip_special_tokens=True).strip()
    
    return answer, lang

print('✅ RAG + Generation functions ready!')

## Step 6: Full Voice Pipeline

এটাই main function — audio in, audio out।

In [ ]:
import time

def full_voice_pipeline(audio_path, tts_gender='female', verbose=True):
    """
    Complete voice pipeline:
    Audio → Whisper STT → RAG → Phi-3.5 → Edge-TTS → Audio
    
    Args:
        audio_path: student এর voice recording (.wav/.mp3)
        tts_gender: 'female' বা 'male' AI voice
        verbose: step by step progress দেখাবে
    
    Returns:
        dict: {question, answer, language, audio_path, timings}
    """
    timings = {}
    
    # ── Step 1: STT ─────────────────────────────────────────────
    t0 = time.time()
    if verbose: print('🎤 Step 1: Audio → Text (Whisper)...')
    
    stt_result = transcribe_audio(audio_path)
    question = stt_result['text']
    detected_lang = stt_result['language']
    
    timings['stt'] = time.time() - t0
    if verbose:
        print(f'   ✅ Transcribed ({timings["stt"]:.1f}s): "{question}"')
        print(f'   🌐 Language detected: {detected_lang}')
    
    # Empty audio check
    if not question.strip():
        return {'error': 'কোনো কথা শুনতে পাইনি। আবার বলো।'}
    
    # ── Step 2: RAG + LLM ────────────────────────────────────────
    t1 = time.time()
    if verbose: print('\n🧠 Step 2: RAG + Phi-3.5 উত্তর generate হচ্ছে...')
    
    answer, answer_lang = generate_answer(question)
    
    timings['llm'] = time.time() - t1
    if verbose:
        print(f'   ✅ Answer generated ({timings["llm"]:.1f}s):')
        print(f'   📝 {answer[:200]}...')
    
    # ── Step 3: TTS ─────────────────────────────────────────────
    t2 = time.time()
    if verbose: print('\n🔊 Step 3: Text → Bengali Audio (Edge-TTS)...')
    
    output_audio = tempfile.mktemp(suffix='.mp3')
    generate_speech(
        answer,
        language=answer_lang,
        gender=tts_gender,
        output_path=output_audio
    )
    
    timings['tts'] = time.time() - t2
    timings['total'] = time.time() - t0
    
    if verbose:
        print(f'   ✅ Audio generated ({timings["tts"]:.1f}s)')
        print(f'\n⚡ Total time: {timings["total"]:.1f}s')
        print(f'   STT: {timings["stt"]:.1f}s | LLM: {timings["llm"]:.1f}s | TTS: {timings["tts"]:.1f}s')
    
    return {
        'question': question,
        'answer': answer,
        'language': answer_lang,
        'audio_path': output_audio,
        'timings': timings
    }

print('✅ Full voice pipeline ready!')

## Step 7: Test — Text Input দিয়ে Pipeline Test

Real audio না থাকলে text থেকে test audio বানিয়ে test করা।

In [ ]:
import IPython.display as ipd

async def create_test_audio(text, path):
    """Test এর জন্য Bengali text থেকে audio বানানো।"""
    communicate = edge_tts.Communicate(text, 'bn-BD-NabanitaNeural')
    await communicate.save(path)

# Test questions
test_questions = [
    'বন্ধু মানে কী?',
    'পরিবারে কারা থাকে?',
]

print('=' * 60)
print('🧪 Full Voice Pipeline Test')
print('=' * 60)

for question in test_questions:
    print(f'\n❓ Test Question: {question}')
    
    # Step 1: Test audio তৈরি করো (student এর voice simulate)
    test_audio_path = tempfile.mktemp(suffix='.mp3')
    asyncio.run(create_test_audio(question, test_audio_path))
    print(f'   🎤 Test audio created')
    
    # Step 2: Full pipeline run করো
    result = full_voice_pipeline(test_audio_path, verbose=True)
    
    if 'error' not in result:
        print(f'\n📢 AI এর উত্তর (audio):')
        display(play_audio(result['audio_path']))
        print(f'📝 Text: {result["answer"]}')
    else:
        print(f'❌ Error: {result["error"]}')
    
    print('=' * 60)

## Step 8: Real Audio File দিয়ে Test

তোমার নিজের voice recording দিয়ে test করো।

In [ ]:
# ── Option A: Kaggle এ audio file upload করে test ──────────────
# /kaggle/input/ এ audio file upload করো
# তারপর নিচের path দাও:

MY_AUDIO = '/kaggle/input/your-audio/question.wav'  # ← তোমার audio path

if os.path.exists(MY_AUDIO):
    print('🎤 তোমার audio দিয়ে test হচ্ছে...')
    result = full_voice_pipeline(MY_AUDIO)
    
    print(f'\n❓ তুমি বলেছো: {result["question"]}')
    print(f'🤖 AI উত্তর: {result["answer"]}')
    display(play_audio(result['audio_path']))

else:
    print('⚠️  Audio file পাওয়া যায়নি।')
    print('   Kaggle এ audio file upload করে path দাও।')
    print('   অথবা Step 7 এর test দিয়ে pipeline check করো।')

## Step 9: Django API এর জন্য Pipeline Function

এই function টা Django view এ directly use করা যাবে।

In [ ]:
import base64

def process_voice_request(audio_bytes, session_id=None, gender='female'):
    """
    Django REST API request handle করার function।
    
    Input:  audio bytes (Flutter থেকে আসবে)
    Output: JSON response (text + audio base64)
    
    Django view এ এভাবে use করবে:
    
    @api_view(['POST'])
    def ai_tutor_chat(request):
        audio_bytes = request.data['audio']
        response = process_voice_request(audio_bytes)
        return Response(response)
    """
    # Audio bytes → temp file
    audio_path = tempfile.mktemp(suffix='.wav')
    with open(audio_path, 'wb') as f:
        if isinstance(audio_bytes, str):  # base64 string
            f.write(base64.b64decode(audio_bytes))
        else:  # raw bytes
            f.write(audio_bytes)
    
    # Pipeline run করো
    result = full_voice_pipeline(audio_path, tts_gender=gender, verbose=False)
    
    if 'error' in result:
        return {
            'success': False,
            'error': result['error']
        }
    
    # Response audio → base64 (Flutter এ পাঠানোর জন্য)
    with open(result['audio_path'], 'rb') as f:
        audio_base64 = base64.b64encode(f.read()).decode('utf-8')
    
    return {
        'success': True,
        'session_id': session_id,
        'question': result['question'],           # Whisper transcription
        'answer_text': result['answer'],          # Phi-3.5 answer
        'answer_audio': audio_base64,             # Edge-TTS audio (base64)
        'language': result['language'],           # 'bengali' বা 'english'
        'timings': result['timings']             # Performance metrics
    }

print('✅ Django-ready API function তৈরি!')
print('\n📋 API Response Format:')
print('''
{
  "success": true,
  "session_id": "...",
  "question": "বন্ধু মানে কী?",          ← Whisper STT
  "answer_text": "বন্ধু হলো...",         ← Phi-3.5 RAG
  "answer_audio": "base64...",           ← Edge-TTS
  "language": "bengali",
  "timings": {"stt": 2.1, "llm": 7.5, "tts": 1.2, "total": 10.8}
}
''')

## ✅ Summary

In [ ]:
print('=' * 60)
print('✅ Voice Pipeline — Complete!')
print('=' * 60)
print('''
🎤 STT  : Whisper Medium — Bengali + English
🧠 LLM  : Phi-3.5 Mini 4-bit — NCTB RAG
🔊 TTS  : Edge-TTS — bn-BD-NabanitaNeural
⚡ Speed : ~10-12s total per response

Expected latency breakdown:
  Whisper STT  : ~2-3s
  Phi-3.5 RAG  : ~7-8s  
  Edge-TTS     : ~1-2s
  ─────────────────────
  Total        : ~10-13s

🚀 পরবর্তী step:
  → Django REST API তে integrate করা
  → Flutter app থেকে audio send/receive
  → Frustration detection add করা
  → Quiz mode integrate করা
''')
print('=' * 60)